## Import Data via Yaml

In [21]:
import yaml
with open(r'postgres.yaml') as file:
    psql = yaml.full_load(file)

In [22]:
psql

{'host': 'localhost', 'database': 'nba', 'user': 'samivanecky', 'port': 5432}

## Import Libraries

In [23]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import json
import nba_api
import requests
import seaborn as sns
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
import psycopg2
from sqlalchemy import create_engine

pd.options.mode.chained_assignment = None 

# Connect to postgres
conn = psycopg2.connect(
    host=psql['host'],
    database=psql['database'],
    user=psql['user'],
    port=psql['port'])

## Data Setup

### Get NBA Teams

In [24]:
# Get teams
nba_teams = teams.get_teams()

In [25]:
# Loop over teams and get games for each team abbreviation (max 30k per team)
for team in nba_teams:
    # If games has already been created
    try:
        gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team['id'])
        # The first DataFrame of those returned is what we want.
        games = games.append(gamefinder.get_data_frames()[0])
    except:
        # Get the games
        gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team['id'])
        # Get games in dataframe format
        games = gamefinder.get_data_frames()[0]

### Variable Manipulation

In [26]:
# Create season and opponent variables
games['SEASON_ID'] = games['SEASON_ID'].str[-4:]
games['OPP'] = games['MATCHUP'].str[-3:]

## Write to Postgres

In [27]:
# Create engine string
connect_str = 'postgresql+psycopg2://' + psql['user'] + '@' + psql['host'] + '/' + psql['database'] 

# Create engine connection
engine = create_engine(connect_str)

c = engine.connect()
conn = c.connection

In [28]:
games.to_sql('games', engine, if_exists='replace')